In [68]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

Откроем файл с данными.

In [69]:
df = pd.read_csv('credit_train.csv')

In [70]:
df.head()

,id,gender,age,marital_status,credit_sum,credit_month,tariff_id,education,credit_count,overdue_credit_count,job_pos,credit_score,region,income,opened
0,26031,M,24,UNM,"24000,00",10,1.60,SCH,2.0,0.0,SPC,"0,728851",ОБЛ ЛЕНИНГРАДСКАЯ,35000.0,True
1,148877,M,23,UNM,"16339,00",18,1.10,SCH,2.0,0.0,SPC,"0,347385",МОСКВА Г,38000.0,True
2,142019,M,54,UNM,"15652,00",10,1.32,GRD,2.0,0.0,SPC,"0,349705",ОБЛ МОСКОВСКАЯ,38000.0,True
3,146816,F,52,UNM,"16560,00",10,1.10,SCH,1.0,0.0,SPC,"0,359232",РЕСП МОРДОВИЯ,28000.0,False
4,27699,F,63,WID,"9078,00",18,1.10,SCH,4.0,0.0,SPC,"0,529368",ЧЕЛЯБИНСКАЯ ОБЛ,30000.0,False


Удалим колонки с которыми мы не умеем работать (ну или вы умеете :D)

In [71]:
y_train = df['opened']
X_train = df.drop(columns=['gender', 'marital_status', 'education', 'job_pos', 'region', 'opened'])
X_train['credit_sum'] = X_train['credit_sum'].apply(lambda x: float(x.replace(',', '.')))
X_train['credit_score'] = X_train['credit_score'].apply(lambda x: float(x.replace(',', '.')))
X_train = X_train.fillna(-1)

Обучим что-нибудь классное

In [73]:
clf = RandomForestClassifier(n_estimators=10, max_depth=15)
print(cross_val_score(clf, X_train, y_train, cv = 5, scoring='f1').mean())
clf.fit(X_train, y_train)

0.15736432502040495


RandomForestClassifier(max_depth=15, n_estimators=10)

Считаем тестовую выборку и предскажем ответы для нее

In [74]:
df_test = pd.read_csv('credit_test.csv')
df_test.head()

,id,gender,age,marital_status,credit_sum,credit_month,tariff_id,education,credit_count,overdue_credit_count,job_pos,credit_score,region,income
0,39162,F,18,UNM,"39469,00",6,1.44,GRD,NaN,NaN,SPC,"0,424628",САНКТ-ПЕТЕРБУРГ Г,30000.0
1,10673,F,63,WID,"16693,00",10,1.32,GRD,2.0,0.0,SPC,"0,311945",ОБЛ ЛЕНИНГРАДСКАЯ,21250.0
2,121409,M,42,MAR,"60260,00",10,1.10,GRD,5.0,0.0,SPC,"0,421982",МОСКВА Г,150000.0
3,23504,M,41,MAR,"32479,00",12,1.10,GRD,1.0,0.0,DIR,"0,406572",ТАТАРСТАН РЕСП,150000.0
4,69269,F,24,UNM,"8159,00",12,1.10,SCH,2.0,0.0,SPC,"0,418243",ТВЕРСКАЯ ОБЛ,16000.0


Удалим колонки с которыми мы не работаем, как и на тренировочной выборке

In [75]:
X_test = df_test.drop(columns=['gender', 'marital_status', 'education', 'job_pos', 'region'])
X_test['credit_sum'] = X_test['credit_sum'].apply(lambda x: float(x.replace(',', '.')))
X_test['credit_score'] = X_test['credit_score'].apply(lambda x: float(x.replace(',', '.')))
X_test = X_test.fillna(-1)

Предскажем значения для тестовой выборки

In [79]:
y_pred = clf.predict(X_test)

In [80]:
y_pred

array([False, False, False, ..., False, False, False])

Создадим отдельный датасет для отправки на kaggle. 

Данный датасет будет содержать только два необходимых столбца — id и наш ответ.

In [81]:
df_kaggle = df_test[['id',]]
df_kaggle['opened'] = y_pred.astype(bool)
df_kaggle.head()

<ipython-input-81-fb2343b59f48>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kaggle['opened'] = y_pred.astype(bool)


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


Сохраним полученный результат в .csv файл для отправки на kaggle

In [82]:
df_kaggle.to_csv('my_submission.csv', index=False)